In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "15008033" # 店舗ID
SPREADSHEET_KEY = '1y4gqQA1ZGelooI1zwqcdKLU1ShKjwhOUSgg3XT_3tcY' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 1000 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "112":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "112":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025868', '1', '5385.0', '-4513.0', '0', '0']
['08/19', '025563', '2', '11538.0', '5719.0', '20', '8']
['08/19', '025523', '3', '3846.0', '603.0', '6', '1']
['08/19', '025771', '5', '8462.0', '904.0', '24', '5']
['08/19', '025774', '6', '769.0', '-1203.0', '0', '0']
['08/19', '025835', '7', '8974.0', '2107.0', '38', '4']
['08/19', '025831', '8', '9487.0', '13543.0', '29', '2']
['08/19', '025647', '10', '4359.0', '2408.0', '10', '1']
['08/19', '025865', '11', '13077.0', '2408.0', '39', '4']
['08/19', '025865', '15', '27692.0', '-3911.0', '62', '12']
['08/19', '025892', '16', '17692.0', '5117.0', '26', '8']
['08/19', '025605', '17', '5385.0', '904.0', '7', '3']
['08/19', '025776', '18', '7179.0', '-2406.0', '11', '2']
['08/19', '025712', '20', '1538.0', '-1203.0', '1', '1']
['08/19', '025862', '21', '2564.0', '2709.0', '8', '1']
['08/19', '025523', '22', '5385.0', '-1805.0', '4', '2']
['08/19', '025863', '23', '6410.0', '904.0', '12', '4']
['08/19', '025590', '25', '10769.0', '

['08/19', '025553', '221', '28718.0', '7223.0', '26', '8']
['08/19', '025553', '222', '19744.0', '-6920.0', '10', '4']
['08/19', '025579', '223', '20256.0', '-7823.0', '4', '1']
['08/19', '025579', '225', '7179.0', '-5416.0', '0', '0']
['08/19', '025579', '226', '17436.0', '3311.0', '9', '2']
['08/19', '025579', '227', '15897.0', '603.0', '7', '2']
['08/19', '025579', '228', '11795.0', '-1203.0', '5', '1']
['08/19', '025877', '230', '30256.0', '-1504.0', '27', '9']
['08/19', '025877', '231', '23846.0', '3913.0', '33', '10']
['08/19', '025877', '232', '18718.0', '8126.0', '31', '11']
['08/19', '025877', '233', '21282.0', '904.0', '23', '6']
['08/19', '025879', '235', '18974.0', '-1203.0', '11', '4']
['08/19', '025818', '236', '3077.0', '-3008.0', '0', '0']
['08/19', '025807', '237', '0', '0', '0', '0']
['08/19', '025824', '238', '7949.0', '27987.0', '58', '4']
['08/19', '025645', '250', '5385.0', '3311.0', '5', '1']
['08/19', '025886', '251', '25641.0', '4515.0', '20', '5']
['08/19', '0

['08/19', '025852', '555', '17179.0', '-902.0', '24', '2']
['08/19', '025779', '556', '5128.0', '-3911.0', '1', '1']
['08/19', '025680', '557', '43590.0', '-19559.0', '12', '7']
['08/19', '025871', '558', '17949.0', '-1805.0', '10', '4']
['08/19', '025791', '560', '23333.0', '5117.0', '15', '1']
['08/19', '025263', '561', '35897.0', '-3309.0', '24', '4']
['08/19', '025864', '562', '8974.0', '-6619.0', '1', '1']
['08/19', '025895', '563', '55641.0', '23473.0', '44', '12']
['08/19', '025665', '565', '51282.0', '-27985.0', '12', '3']
['08/19', '025665', '566', '33077.0', '-23170.0', '5', '4']
['08/19', '025665', '567', '35641.0', '4515.0', '23', '4']
['08/19', '025665', '568', '27436.0', '-9930.0', '9', '3']
['08/19', '025665', '570', '41282.0', '16251.0', '34', '3']
['08/19', '025665', '571', '21538.0', '-1504.0', '11', '1']
['08/19', '025665', '572', '41026.0', '16853.0', '36', '8']
['08/19', '025665', '573', '35897.0', '3010.0', '23', '6']
['08/19', '025665', '575', '31026.0', '-12036.

['08/19', '120088', '1050', '1878', '-185.0', '5', '6', '--']
['08/19', '119962', '1051', '3937', '-1019.0', '9', '10', '--']
['08/19', '119972', '1052', '2501', '-972.0', '5', '4', '--']
['08/19', '120033', '1053', '2951', '-1065.0', '6', '7', '--']
['08/19', '120036', '1055', '2859', '-139.0', '10', '3', '--']
['08/19', '120099', '1056', '4467', '-185.0', '10', '7', '--']
['08/19', '120021', '1057', '4327', '926.0', '32', '15', '--']
['08/19', '120021', '1058', '5424', '556.0', '39', '14', '--']
['08/19', '120021', '1060', '4756', '602.0', '35', '9', '--']
['08/19', '120021', '1061', '2810', '-602.0', '17', '4', '--']
['08/19', '120021', '1062', '4452', '139.0', '29', '15', '--']
['08/19', '120021', '1063', '4601', '-833.0', '28', '10', '--']
['08/19', '120025', '1065', '4795', '139.0', '14', '15', '--']
['08/19', '120025', '1066', '5084', '-231.0', '17', '12', '--']
['08/19', '120025', '1067', '3634', '-648.0', '10', '9', '--']
['08/19', '120025', '1068', '8087', '-93.0', '31', '19'

['08/19', '120125', '1253', '835', '-694.0', '1', '0', '--']
['08/19', '120125', '1255', '1308', '-463.0', '3', '2', '--']
['08/19', '120125', '1256', '5432', '-602.0', '14', '17', '--']
['08/19', '120088', '1257', '3636', '463.0', '11', '9', '--']
['08/19', '120033', '1258', '6448', '2083.0', '27', '10', '--']
['08/19', '120088', '1260', '568', '-139.0', '1', '1', '4']
['08/19', '120033', '1261', '6353', '833.0', '23', '15', '--']
['08/19', '120043', '1262', '5383', '-278.0', '18', '14', '--']
['08/19', '120033', '1263', '3630', '-463.0', '8', '12', '--']
['08/19', '119972', '1265', '1417', '-694.0', '2', '4', '--']
['08/19', '120036', '1266', '1713', '-139.0', '5', '4', '--']
['08/19', '120034', '1267', '1114', '185.0', '0', '1', '8']
['08/19', '120034', '1268', '6774', '1296.0', '0', '10', '59']
['08/19', '120034', '1270', '584', '-648.0', '0', '0', '1']
['08/19', '120034', '1271', '5156', '4444.0', '0', '7', '53']
['08/19', '120034', '1272', '1969', '231.0', '0', '2', '18']
['08/19

['08/19', '120125', '1557', '1315', '-417.0', '3', '3', '--']
['08/19', '120125', '1558', '5060', '1528.0', '22', '19', '--']
['08/19', '120125', '1560', '2028', '-926.0', '5', '3', '--']


{'spreadsheetId': '1y4gqQA1ZGelooI1zwqcdKLU1ShKjwhOUSgg3XT_3tcY',
 'updates': {'spreadsheetId': '1y4gqQA1ZGelooI1zwqcdKLU1ShKjwhOUSgg3XT_3tcY',
  'updatedRange': 'S5RAW!F6001:M6076',
  'updatedRows': 76,
  'updatedColumns': 8,
  'updatedCells': 608}}